In [1]:
import torch
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/judd/Documents/optimisation_of_superconduncting_circuits/core')
from zeropi import ZeroPi
import general as general
import scqubits as sc


In [2]:
#Validating T2 Calcs Against SCQUBITS
EJ = torch.tensor(10.00, requires_grad=True, dtype=torch.double)
EL = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
ECJ = torch.tensor(20, requires_grad=True, dtype=torch.double)
EC = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
dEJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
dCJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
flux = torch.tensor(0.23, requires_grad=True, dtype=torch.double)
ng = 0.1
ncut = 30
truncated_dim = 10
pt_count = 10
min_val = -19
max_val = 19
hamiltonian_creation = 'create_H'

zeropi = ZeroPi(EJ=EJ, 
                 EL=EL , 
                 ECJ=ECJ, 
                 EC=EC, 
                 dEJ=dEJ, 
                 dCJ=dCJ, 
                 flux=flux, 
                 ng=ng, 
                 ncut=ncut, 
                 truncated_dim=truncated_dim, 
                 pt_count=pt_count, 
                 min_val=min_val, 
                 max_val=max_val,
                 hamiltonian_creation=hamiltonian_creation)

zeropi_sc = sc.ZeroPi(
            grid = sc.Grid1d(min_val=min_val, max_val=max_val, pt_count=pt_count),
            EJ   = EJ.item(),
            EL   = EL.item(), 
            ECJ  = ECJ.item(),
            EC   = EC.item(),
            ng   = ng,
            flux = flux.item(),
            ncut = ncut, 
            dEJ = dEJ.item(), 
            dCJ = dCJ.item())

print(zeropi_sc.t1_inductive() *  general.effective_t1_rate(zeropi , "t1_inductive").item())
print(zeropi_sc.t1_flux_bias_line() *  general.effective_t1_rate(zeropi , "t1_flux_bias_line").item())
print(zeropi_sc.tphi_1_over_f_cc()  *  general.effective_tphi_rate(zeropi , "tphi_1_over_f_cc").item())
print(zeropi_sc.tphi_1_over_f_flux() * general.effective_tphi_rate(zeropi , "tphi_1_over_f_flux").item())

print('____')

print(zeropi_sc.t1_flux_bias_line())
print( 1/general.effective_t1_rate(zeropi , "t1_flux_bias_line").item())


See documentation for details.
This warning can be disabled by executing:
scqubits.settings.T1_DEFAULT_WARNING=False

 /Users/judd/Documents/optimisation_of_superconduncting_circuits/venv/lib/python3.8/site-packages/scqubits/core/noise.py: 1195

TypeError: band_matrix() takes 5 positional arguments but 6 were given

In [ ]:
general.effective_t1_rate(zeropi , "t1_inductive")
general.effective_tphi_rate(zeropi , zeropi.tphi_supported_noise_channels())
general.t2_rate(zeropi , zeropi.t1_supported_noise_channels(), zeropi.tphi_supported_noise_channels())

tensor([[2.6648e-05]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
#Optimisation

#VARIABLE PARAMS
EJ = torch.tensor(10.00, requires_grad=True, dtype=torch.double)
EL = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
ECJ = torch.tensor(20, requires_grad=True, dtype=torch.double)
EC = torch.tensor(0.04, requires_grad=True, dtype=torch.double)
dEJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
dCJ = torch.tensor(0.0, requires_grad=True, dtype=torch.double)
flux = torch.tensor(0.23, requires_grad=True, dtype=torch.double)

#FIXED PARAMS
ng = 0.1
ncut = 30
truncated_dim = 10
pt_count = 10
min_val = -19
max_val = 19
hamiltonian_creation = 'auto_H'

zeropi = ZeroPi(EJ=EJ, 
                 EL=EL , 
                 ECJ=ECJ, 
                 EC=EC, 
                 dEJ=dEJ, 
                 dCJ=dCJ, 
                 flux=flux, 
                 ng=ng, 
                 ncut=ncut, 
                 truncated_dim=truncated_dim, 
                 pt_count=pt_count, 
                 min_val=min_val, 
                 max_val=max_val,
                 hamiltonian_creation=hamiltonian_creation)

#RECORDING VALUES
EJ_vals = [EJ.item()]
EC_vals = [EC.item()]
EL_vals = [EL.item()]
R2_rate = [1e100, 1e99]

# GRADIENT DESCENT
i = 0
while R2_rate[-1] < R2_rate[-2]:
    i+=1

    fluxonium = ZeroPi(EJ, EC, EL, flux, dim, 'create_H')
    R2  = general.t2_rate(fluxonium, fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels())
    R2_rate.append(R2.item())

    R2.backward()
    with torch.no_grad():
        #If paramater value drops below lower bound, it will remain on the lower bound
        ##WHY DO WE - the gradient and not add?
        EJ.data = EJ - EJ.grad*learn_rate if EJ - EJ.grad*learn_rate > EJ_bounds[0] else torch.tensor(EJ_bounds[0])
        EL.data = EL - EL.grad*learn_rate if EL - EL.grad*learn_rate > EL_bounds[0] else torch.tensor(EL_bounds[0])
        EC.data = EC - EC.grad*learn_rate if EC - EC.grad*learn_rate > EC_bounds[0] else torch.tensor(EC_bounds[0])
    
    #RECORDING VALUES FOR PLOT
    EJ_vals.append(EJ.item())
    EC_vals.append(EC.item())
    EL_vals.append(EL.item())

        
    print(f"{i}: R2: {R2.item()}, EJ: {EJ.item()}, EL: {EL.item()}, EC: {EC.item()}")
